In [17]:
import pandas as pd
import os

In [18]:
class company_prerocessing():
#     def __init__()
    # 파일 열기
    def open_df(folder_path):
        df = pd.read_csv(folder_path)
        return df

    # 재무 상태표
    def bs_preprocessing(df_bs, cmp_list):
        # 행 3번째가 숫자면 0, 2, 3, 4 행만 가져오기
        if df_bs.columns[2].isdigit() == True:
            try:
                df_bs_1 = df_bs.iloc[:, [0, 2, 3, 4]]
            except:
                pass
         # 행 6번째가 숫자면 1, 6, 7, 8 행만 가져오기
        elif df_bs.columns[6].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]
         # 행 7번째가 숫자면 1, 7, 8, 9 행만 가져오기
        elif df_bs.columns[7].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
         # 행 8번째가 숫자면 1, 8, 9, 10 행만 가져오기
        elif df_bs.columns[8].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]
        # 위에 모든 조건이 맞지 않다면 0:4 가져오기
        else:
            df_bs_1 = df_bs.iloc[:, 0:4]
        # 컬럼 이름 지정
        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        # 데이터 프레임 전치
        df_bs_1 = df_bs_1.T
        
        # 행 이름을 0번째 줄로 바꿈
        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        # 0번째 열 제거
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        # 'label_ko' 행 제거
        df_bs_1.drop('label_ko', inplace=True, axis =1)
        # 인덱스 리셋, index 행이름 바꾸기
        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})
        
        # company  행 추가 후 회사 이름 넣기
        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')

        return df_bs_1

    # 포괄손익 계산서
    def cis_preprocessing(df_cis, cmp_list):
        
        # 6번째 행에 '01-20'이 포함되고 8번째 행에 '01-20'이 포함되어 있을 때(3년치 데이터가 있을 시)
        if '01-20' in df_cis.columns[6]:
            if '01-20' in df_cis.columns[8]:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7, 8]]
        # 3년치 데이터가 없을 때 2년치 데이터만 긁어옴
            else:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7]]
        # 7번째 행에 '01-20'이 포함되고 9번째 행에 '01-20'이 포함되어 있을 때(3년치 데이터가 있을 시)
        elif '01-20' in df_cis.columns[7]:
            if '01-20' in df_cis.columns[9]:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
            # 3년치 데이터가 없을 때 2년치 데이터만 긁어옴
            else:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8]]
        
        # 데이터 프레임의 행 수가 4개면 행 이름을 아래와 같이 변경
        if len(df_cis_1.columns) == 4:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2], df_cis_1.columns[3]]
        # 데이터 프레임의 행 수가 3개면 행 이름을 아래와 같이 변경
        elif len(df_cis_1.columns) == 3:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]
        
        # 데이터 프레임 전치
        df_cis_1 = df_cis_1.T
        
        # 행 이름을 0번째 줄로 바꿈
        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        # 0번째 열 제거
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        # 'label_ko' 행 제거
        df_cis_1.drop('label_ko', inplace=True, axis =1)
        # 인덱스 리셋, index 행이름 바꾸기
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
        
        # 인덱스 리셋, index 행이름 바꾸기
        df_cis_1['company'] = cmp_list
        # date의 9번째 줄 부터 끝까지 바꾸기
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        
        try:
            if '관리비' and '판매비' in df_cis_1.columns:
                df_cis_1['관리비'] = df_cis_1['관리비'].astype('float')
                df_cis_1['판매비'] = df_cis_1['판매비'].astype('float')
                df_cis_1['판매비와관리비'] = df_cis_1['관리비'] + df_cis_1['판매비']
                df_cis_1.drop(['관리비', '판매비'], inplace=True, axis=1)
        except:
            
            pass
        
        print('cis_preprocessing complete')

        return df_cis_1
    # 데이터 프레임 저장
    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)

In [19]:
# if '__main__' == __name__:

FOLDER_PATH = 'data/'
# 원본 PATH
PATH1 = '최종/건실기업_재무상태표/'
PATH2 = '최종/건실기업_포괄손익계산서/'

#전처리 PATH
PATH3 = '최종/건실기업_재무상태표 전처리/'
PATH4 = '최종/건실기업_포괄손익계산서 전처리/'

# 파일 리스트화
file_list1 = os.listdir(FOLDER_PATH + PATH1)
file_list2 = os.listdir(FOLDER_PATH + PATH2)

# 변수 지정
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출원가':4, '매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10, '판매비와관리비':7}

# 빈 데이터프레임 생성
bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

for cmp_list1, cmp_list2 in zip(file_list1, file_list2):
    print(cmp_list1, cmp_list2)
    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH1 + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), df_bs_1, cmp_list1[:-13])

    # 변수 전처리
    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ == len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])
    
    # 재무상태표 데이터프레임 합치기
    bs_a = pd.concat([bs_a, df_bs_1[list1]])

    # 포괄손익 계산서
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH2+ cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), df_cis_1, cmp_list2[:-15])

    # 변수 전처리
    list2 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list2.append(cis)
    print(df_cis_1[list2])
    
    # 포괄손익 계산서 데이터프레임 합치기
    cis_a = pd.concat([cis_a, df_cis_1[list2]])


# 인덱스 리셋 및 저장
bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), cis_a, '포괄손익계산서 통합')


DMS 재무상태표 원본.csv DMS 포괄손익계산서 원본.csv
bs_preprocessing complete
       date company            자본총계            자산총계  매입채무 및 기타유동채무  \
0  20191231     DMS  166005674664.0  302514623836.0  32096890971.0   
1  20181231     DMS  151479426195.0  317314698057.0  39977684190.0   
2  20171231     DMS  135202873362.0  298505199522.0  69184418522.0   

   매출채권 및 기타유동채권            부채총계            유동부채            유동자산  \
0  26620302586.0  136508949172.0  135345962490.0  134517297914.0   
1  47088416662.0  165835271862.0  162426446113.0  159939971787.0   
2  25987507374.0  163302326160.0  146972021897.0  149549936171.0   

            재고자산  
0  14498465305.0  
1  14616273960.0  
2  17969335446.0  
cis_preprocessing complete
       date company            매출원가          매출총이익 법인세비용차감전순이익(손실)  \
0  20191231     DMS  130906502366.0  65300086704.0   21544765028.0   
1  20181231     DMS  222750210188.0  75967702026.0   21868026243.0   
2  20171231     DMS  207523598515.0  61208001998.0    4835649098.0   



In [22]:
# 전기자산, 자본 컬럼 만듬
bs_a['전기자산총계'] = 0
bs_a['전기자본총계'] = 0

In [23]:
# 전기 데이터 넣음 ( 마지막년도는 없어지기 때문에 상관없이 넣을 수 있음)
for i in range(len(bs_a) - 1):
    bs_a['전기자산총계'][i] = bs_a['자산총계'][i+1]
    bs_a['전기자본총계'][i] = bs_a['자본총계'][i+1]

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the

In [24]:
result = pd.merge(bs_a, cis_a, on=['date','company'])

In [55]:
# 인덱스를 date로 만들고 마지막 년도인 20171231을 드랍시킵
result.set_index(result['date'], drop=True, inplace=True)
result = result.drop('20171231')

In [63]:
#인덱스 초기화
result.reset_index(drop=True, inplace=True)

In [64]:
# 널값 제거
result.dropna(inplace=True)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
# 날짜값 제거
result.drop(['date'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [67]:
result.to_csv('data/건실기업.csv', index=False)